# DeepVO

Training and testing script for colaborate ultra

Check GPU and Import modules

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

%matplotlib !pip uninstall -y Pillow

# install the new one
!pip install Pillow==5.3.0
# import the new one
import PIL
print(PIL.PILLOW_VERSION)inline

 http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

# load tqdm
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip

from PIL import Image
from tqdm.autonotebook import tqdm

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import random
import pickle
import gzip
import io
import os
import os.path
import time
import copy
import shutil
import math
import datetime

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.utils.data as data_utils
import torch.nn.functional as F

from torch.nn.init import calculate_gain
from torch.autograd import Variable
from torch.autograd import Function

import torchvision
from torchvision import transforms
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms

Mount data folders 

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Build training and validation data

Set parameters

In [ ]:
args ={"datapath":'drive/My Drive/VOdata/alex_3small_loops', "checkpoint_path":'./Checkpoint',
       "checkpoint":None,
       "bsize":32, "lr":0.001, "weight_decay":1e-4, "trajectory_length":5, "dropout_p":0.85,
       "resize":64, "K":100, "epochs"=100, "patience"=40}

if not os.path.exists(args["checkpoint_path"]): os.makedirs(args["checkpoint_path"]) 

model = DeepVONet(args["resize"], args["resize"], args["dropout_p"])

print("Number of parameters:", human_format(count_parameters(model)))
print("Number of parameter bytes:", human_format(32*count_parameters(model)))

USE_GPU = torch.cuda.is_available()

if USE_GPU:
    model.cuda()

if args["checkpoint"] is not None:
    the_checkpoint = torch.load(args["checkpoint"])
    model.load_state_dict(the_checkpoint['state_dict'])

Train 

In [ ]:
best_model, logs, args_ = train(model, preprocess, args, epochs=args["epochs"], patience=args["patience"])
plot_train_valid(logs, args_)

Test

In [ ]:
test_loss, predicted_test_relative_poses = test(best_model, preprocess, args_)
test_data = DeepVOdata(datapath=args["datapath"], trajectory_length=args["trajectory_length"],
                           transform=preprocess, kind='test')